# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ffb94e9bbe0>
├── 'a' --> tensor([[ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970]])
└── 'x' --> <FastTreeValue 0x7ffb94e9bc70>
    └── 'c' --> tensor([[-2.0211,  0.5310, -0.0415,  0.4631],
                        [-0.4067,  0.4376,  1.6817,  0.2186],
                        [-0.7215, -1.5169, -0.5356, -0.3247]])

In [4]:
t.a

tensor([[ 1.3636,  1.6924, -0.1848],
        [ 1.0244, -0.0365,  0.2970]])

In [5]:
%timeit t.a

74.7 ns ± 1.47 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ffb94e9bbe0>
├── 'a' --> tensor([[-0.0428, -1.1768,  0.0802],
│                   [ 0.6637,  1.4329,  1.5996]])
└── 'x' --> <FastTreeValue 0x7ffb94e9bc70>
    └── 'c' --> tensor([[-2.0211,  0.5310, -0.0415,  0.4631],
                        [-0.4067,  0.4376,  1.6817,  0.2186],
                        [-0.7215, -1.5169, -0.5356, -0.3247]])

In [7]:
%timeit t.a = new_value

76.7 ns ± 0.555 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3636,  1.6924, -0.1848],
               [ 1.0244, -0.0365,  0.2970]]),
    x: Batch(
           c: tensor([[-2.0211,  0.5310, -0.0415,  0.4631],
                      [-0.4067,  0.4376,  1.6817,  0.2186],
                      [-0.7215, -1.5169, -0.5356, -0.3247]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3636,  1.6924, -0.1848],
        [ 1.0244, -0.0365,  0.2970]])

In [11]:
%timeit b.a

66.1 ns ± 1.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2272, -0.3267,  0.0637],
               [-1.3723,  0.6658, -1.1822]]),
    x: Batch(
           c: tensor([[-2.0211,  0.5310, -0.0415,  0.4631],
                      [-0.4067,  0.4376,  1.6817,  0.2186],
                      [-0.7215, -1.5169, -0.5356, -0.3247]]),
       ),
)

In [13]:
%timeit b.a = new_value

589 ns ± 13.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

964 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

279 µs ± 5.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

263 µs ± 5.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ffae7bfd1c0>
├── 'a' --> tensor([[[ 1.3636,  1.6924, -0.1848],
│                    [ 1.0244, -0.0365,  0.2970]],
│           
│                   [[ 1.3636,  1.6924, -0.1848],
│                    [ 1.0244, -0.0365,  0.2970]],
│           
│                   [[ 1.3636,  1.6924, -0.1848],
│                    [ 1.0244, -0.0365,  0.2970]],
│           
│                   [[ 1.3636,  1.6924, -0.1848],
│                    [ 1.0244, -0.0365,  0.2970]],
│           
│                   [[ 1.3636,  1.6924, -0.1848],
│                    [ 1.0244, -0.0365,  0.2970]],
│           
│                   [[ 1.3636,  1.6924, -0.1848],
│                    [ 1.0244, -0.0365,  0.2970]],
│           
│                   [[ 1.3636,  1.6924, -0.1848],
│                    [ 1.0244, -0.0365,  0.2970]],
│           
│                   [[ 1.3636,  1.6924, -0.1848],
│                    [ 1.0244, -0.0365,  0.2970]]])
└── 'x' --> <FastTreeValue 0x7ffae7b0aa60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42.5 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ffb70381400>
├── 'a' --> tensor([[ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970],
│                   [ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970],
│                   [ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970],
│                   [ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970],
│                   [ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970],
│                   [ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970],
│                   [ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970],
│                   [ 1.3636,  1.6924, -0.1848],
│                   [ 1.0244, -0.0365,  0.2970]])
└── 'x' --> <FastTreeValue 0x7ffb703817f0>
    └── 'c' --> tensor([[-2.0211,  0.5310, -0.0415,  0.4631],
                        [-0.4067,  0.4376,  1.6817,  0.2186],
                 

In [23]:
%timeit t_cat(trees)

37.5 µs ± 893 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.7 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.0211,  0.5310, -0.0415,  0.4631],
                       [-0.4067,  0.4376,  1.6817,  0.2186],
                       [-0.7215, -1.5169, -0.5356, -0.3247]],
              
                      [[-2.0211,  0.5310, -0.0415,  0.4631],
                       [-0.4067,  0.4376,  1.6817,  0.2186],
                       [-0.7215, -1.5169, -0.5356, -0.3247]],
              
                      [[-2.0211,  0.5310, -0.0415,  0.4631],
                       [-0.4067,  0.4376,  1.6817,  0.2186],
                       [-0.7215, -1.5169, -0.5356, -0.3247]],
              
                      [[-2.0211,  0.5310, -0.0415,  0.4631],
                       [-0.4067,  0.4376,  1.6817,  0.2186],
                       [-0.7215, -1.5169, -0.5356, -0.3247]],
              
                      [[-2.0211,  0.5310, -0.0415,  0.4631],
                       [-0.4067,  0.4376,  1.6817,  0.2186],
                       [-0.7215, -1.5169, -0.5356, -0.3247]],

In [26]:
%timeit Batch.stack(batches)

98.7 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.0211,  0.5310, -0.0415,  0.4631],
                      [-0.4067,  0.4376,  1.6817,  0.2186],
                      [-0.7215, -1.5169, -0.5356, -0.3247],
                      [-2.0211,  0.5310, -0.0415,  0.4631],
                      [-0.4067,  0.4376,  1.6817,  0.2186],
                      [-0.7215, -1.5169, -0.5356, -0.3247],
                      [-2.0211,  0.5310, -0.0415,  0.4631],
                      [-0.4067,  0.4376,  1.6817,  0.2186],
                      [-0.7215, -1.5169, -0.5356, -0.3247],
                      [-2.0211,  0.5310, -0.0415,  0.4631],
                      [-0.4067,  0.4376,  1.6817,  0.2186],
                      [-0.7215, -1.5169, -0.5356, -0.3247],
                      [-2.0211,  0.5310, -0.0415,  0.4631],
                      [-0.4067,  0.4376,  1.6817,  0.2186],
                      [-0.7215, -1.5169, -0.5356, -0.3247],
                      [-2.0211,  0.5310, -0.0415,  0.4631],
                   

In [28]:
%timeit Batch.cat(batches)

186 µs ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

587 µs ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
